In [32]:
import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib

In [33]:
#Import Azure ML SDK modules
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

In [34]:
from azureml.core import Workspace
ws = Workspace.from_config()
print(ws)


Workspace.create(name='sandbox_ml_service_workspace', subscription_id='72d4c1b3-fd6e-432a-b18a-89eec2c10a6b', resource_group='pep-gdaentml-cus-01-rg')


In [35]:
#creating experiment
exp = Experiment(workspace=ws, name='salary_exp_azureml')

In [36]:
#we will start logging matrics
run = exp.start_logging()                   
run.log("Experiment start time", str(datetime.datetime.now()))

In [37]:
#load , split and train the model
sal = pd.read_csv('data/sal.csv',header=0, index_col=None)
X = sal[['x']]
y = sal['y']
#spliting
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

#train the model
lm = LinearRegression() 
lm.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [38]:
#Freeze the model
filename = 'outputs/sal_model.pkl'
joblib.dump(lm, filename)

['outputs/sal_model.pkl']

In [39]:
#testing the model
filename = 'outputs/sal_model.pkl'
loaded_model=joblib.load(filename)
y=loaded_model.predict([[21]])[0]
print(y)

141760.56910569107


In [40]:
#Log metrics to Azure ML Experiment
run.log('Intercept :', lm.intercept_)
run.log('Slope :', lm.coef_[0])

In [41]:
#end the experiment
run.log("Experiment end time", str(datetime.datetime.now()))
run.complete()

In [42]:
print(run.get_portal_url())

https://ml.azure.com/runs/06db8b64-fe30-499c-877e-0900490c1c49?wsid=/subscriptions/72d4c1b3-fd6e-432a-b18a-89eec2c10a6b/resourcegroups/pep-gdaentml-cus-01-rg/workspaces/sandbox_ml_service_workspace&tid=42cc3295-cd0e-449c-b98e-5ce5b560c1d3


In [43]:
#Register the model
model = Model.register(model_path = "outputs/sal_model.pkl",
                       model_name = "sal_model",
                       tags = {"key": "1"},
                       description = "Salary Prediction",
                       workspace = ws)

Registering model sal_model


In [44]:
# Define Azure ML Deploymemt configuration
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Salary",  "method" : "sklearn"}, 
                                               description='Predict Stackoverflow Salary')

In [45]:
# Create enviroment configuration file
#salenv = CondaDependencies()
#salenv.add_conda_package("scikit-learn")

#with open("salenv.yml","w") as f:
#    f.write(salenv.serialize_to_string())
#with open("salenv.yml","r") as f:
#    print(f.read())

In [46]:
#cell 18 is not helping so again creating new env
# create environment for the deploy
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import AciWebservice

# get a curated environment
env = Environment.get(
    workspace=ws, 
    name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu",
    version=1
)
env.inferencing_stack_version='latest'

# create deployment config i.e. compute resources
#its created in cell 17


Create Azure ML Scoring file

In [49]:

%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('sal_model')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Overwriting score.py


Deploy the model to Azure Container Instance

In [50]:
%%time
#deploying the Model
import uuid
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.model import Model

# get the registered model
model = Model(ws, "sal_model")

# create an inference config i.e. the scoring script and environment
inference_config = InferenceConfig(entry_script="score.py", environment=env)

# deploy the service
service_name = "sal-model-svc-" + str(uuid.uuid4())[:4]
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=aciconfig,
)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-05-02 22:20:03+00:00 Creating Container Registry if not exists.
2022-05-02 22:20:04+00:00 Registering the environment.
2022-05-02 22:20:04+00:00 Use the existing image.
2022-05-02 22:20:04+00:00 Generating deployment configuration.
2022-05-02 22:20:05+00:00 Submitting deployment to compute.
2022-05-02 22:20:10+00:00 Checking the status of deployment sal-model-svc-28be..
2022-05-02 22:22:07+00:00 Checking the status of inference endpoint sal-model-svc-28be.
Succeeded
ACI service creation operation finished, operation "Succeeded"
CPU times: user 879 ms, sys: 163 ms, total: 1.04 s
Wall time: 2min 10s


In [51]:
#test this from endpoint consume code
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {"data":[[11]]
}

body = str.encode(json.dumps(data))

url = 'http://012b3378-15d4-422d-bcc8-574628b4af95.eastus.azurecontainer.io/score'
api_key = '' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


b'"[123358.80758807588]"'
